# IOManager

The **IOManager** class provides several easy-to-use functions for reading and writing ROOT objects.
First, let's create a **TFile** called ```testsample.root``` with a **TTree** called ```TestTree``` containg some dummy data for 10 branches with 10,000 events.

In [ ]:
import ROOT
from mephisto import IOManager

IOManager.CreateTestSample("testsample.root", tree="TestTree", nbranches=10, nevents=1e4)

The new ROOT file should live in your current working directory:

In [ ]:
%ls -l

The branches have the the invenvite names ```branch_X``` where ```X``` ranges from 1 to 10. The events sampled for each branch follows a $\chi^2$-distribution with a mean around ```X```.

Let's fill some histograms:
* **h1** - 1D histogram showing the raw distribution of ```branch_3```
* **h2** - 1D histogram showing the distribution of ```branch_3``` for events passing the ```branch_6>4 && branch_7<9``` selection and weighted by an per-event-weight given by ```branch_10```
* **h3** - 2D histogram plotting the distribution of ```branch_3``` on the x-axis versus the ```branch_4``` on the y-axis

In [ ]:
h1 = ROOT.TH1D("h1", "", 100, 0.0, 10.0)
h2 = ROOT.TH1D("h2", "", 100, 0.0, 10.0)
h3 = ROOT.TH2D("h3", "", 100, 0.0, 10.0, 100, 0.0, 10.0)

h1.Sumw2()
h2.Sumw2()
h3.Sumw2()

IOManager.FillHistogram(h1, "testsample.root", tree="TestTree", varexp="branch_3")
IOManager.FillHistogram(h2, "testsample.root", tree="TestTree", varexp="branch_3",
                        cuts=["branch_6>4", "branch_7<9"], weight="branch_10")
IOManager.FillHistogram(h3, "testsample.root", tree="TestTree", varexp="branch_3:branch_4")

And now let's look at them, by dooing it the (good old) ROOT way:

In [ ]:
c = ROOT.TCanvas()
c.Draw()
h1.Draw("HIST")

In [ ]:
c.Draw()
h2.Draw("HIST")

In [ ]:
c.Draw()
h3.Draw("COL")

You can also add the projection of the ```TestTree``` to a existing histogram without overwriting it using the ```append``` flag:

In [ ]:
print "h1 nentries:".ljust(15), h1.GetEntries()
print "h2 nentries:".ljust(15), h2.GetEntries()

IOManager.FillHistogram(h2, "testsample.root", tree="TestTree", varexp="branch_3", append=True)

print "".ljust(23, "-"), "\nsum:".ljust(15), h2.GetEntries()

Another useful feature is the **Factory** subclass of the IOManager. It allows filling multiple histograms from one TTree in one go!

Setting it up is easy:

In [ ]:
histos = {}  # dictionary of TH1Ds

factory = IOManager.Factory("testsample.root", "TestTree")

for i in range(1, 11):
    varexp = "branch_{}".format(i)
    for j in range(1, 11):
        weight = "branch_{}".format(j)
        for k in range(1, 11):
            cuts = ["branch_{}>{}".format(k, float(i)/k)]
            histos[i, j, k] = ROOT.TH1D("{}_{}_{}".format(i, j, k), "", 100, 0.0, 10.0)
            factory.Register(histos[i, j, k], varexp=varexp, cuts=cuts, weight=weight)
            
factory.Run()

print "i".ljust(5), "j".ljust(5), "k".ljust(5), "nentries\n", "".ljust(26, "-") 
for (i, j, k), histo in histos.items():
    print str(i).ljust(5), str(j).ljust(5), str(k).ljust(5), histo.GetEntries()

Now that you know the basic features of the IOManager check out the other notebooks, too!